# Uttarakhand Industrial Information System

Last Modified: 12/03/2021

Author: Manaruchi Mohapatra

### Import Required Libraries

In [1]:
import dash
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_daq as daq
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import os
import dash_table
import base64
import smtplib, ssl

### Files Setup

In [2]:
cur_path = os.getcwd()
#---------------Paths------------------------------------------------------
area_list_csv_path = os.path.join(cur_path, 'support_files', 'area_list.csv')
#--------------------------------------------------------------------------

with open(os.path.join(cur_path, 'json', 'DIS_Merged_for_Dash.geojson')) as f:
    dis_json = json.load(f)

#Data for main basic map
df = pd.read_csv(os.path.join(cur_path, 'json', 'data.csv'))

#Data for search dropdown content
area_list_df = pd.read_csv(area_list_csv_path)
area_list = area_list_df['label']

#Data for basemaps list
basemap_list_df = pd.read_csv(os.path.join(cur_path, 'support_files', 'basemap_list.csv'))
basemap_list_dict = [{'label': r[0], 'value': r[1]} for i,r in basemap_list_df.iterrows()]

#Data for Plot Details
plot_details_df = pd.read_csv(os.path.join(cur_path, 'support_files', 'plot_details.csv'))
plot_details_col_list = plot_details_df.columns
plot_details_UID_list = plot_details_df['UID'].values

#Data for Plot Details - Administrator Mode
plot_details_admin_file = os.path.join(cur_path, 'support_files', 'test.csv')
plot_details_admin_df = pd.read_csv(plot_details_admin_file)
for c in plot_details_admin_df.columns:
    if(c[:7] == 'Unnamed'):
        plot_details_admin_df = plot_details_admin_df.drop([c],axis = 1)
        
#Data for User Info
user_info_csv_path = os.path.join(cur_path, 'support_files', 'user_list.csv')

#Graphics Folder
graphics_folder_path = os.path.join(cur_path, 'graphics')

## Setup for SMTP

In [3]:
port = 465  # For SSL
password = "RBS@1234"

sender_email = "jjjhkrmntest@gmail.com"

### Dash App Definition

In [4]:
FONT_AWESOME = "https://use.fontawesome.com/releases/v5.7.2/css/all.css"

app = dash.Dash(__name__,external_stylesheets=[dbc.themes.COSMO, FONT_AWESOME,
                                              'https://fonts.googleapis.com/css2?family=Roboto&display=swap'], update_title=None)

app.title = "Uttarakhand Industrial Information System"

## Global Variables

In [5]:
is_logged_in = False
session_user_name = ''
user_info_dict = {'First Name': '',
                  'Middle Name': '',
                  'Last Name': '',
                  'DOB': ''}

## App Layout

In [6]:




header_elements = [
                html.Div(html.H1('Uttarakhand Industrial Information System', className = 'navbar-brand'), className = 'navbar-brand-bar'),
                html.Div([
                    
                    html.Img(src = 'http://www.rbasedservices.com/wp-content/uploads/2020/06/cropped-RBS_LOGO.png', className = 'navbar-logo', id = 'rbs-logo'),
                    html.Img(src = 'https://www.siidcul.com/upload/header/SIIDCUL.jpg', className = 'navbar-logo'),
                    html.Img(src = 'https://www.saferspaces.org.za/cache/ce_img_cache/local/520d1943832aab5c/German-Cooperation-GIZ-topdown1_270_232_80.jpg', className = 'navbar-logo', id = 'giz-logo'),
                    html.Img(src = 'https://www.siidcul.com/upload/header/SIIDCUL.jpg', className = 'navbar-logo', id = 'siidcul-logo'),
                    dbc.Tooltip("State Industrial Development Corporation of Uttarakhand Limited", target = 'siidcul-logo'),
                    dbc.Tooltip("Deutsche Gesellschaft für Internationale Zusammenarbeit", target = 'giz-logo'),
                    dbc.Tooltip("RBased Services Pvt. Ltd.", target = 'rbs-logo')
                    
                ], className = 'navbar-logo-bar')
            ]

  

page_layout = html.Div([
    
    dcc.Location(id='url', refresh=False),
    
    html.Div(header_elements, className = 'mainheader'),
    
    html.Div(className = 'navlinks', id='navlinkscallback'),
    
    dbc.Row(no_gutters=True, className = 'mainarea', id='mainareacallback'),
    
], className = 'mainbody')

### Layout for Basic Web Map

In [7]:
fig = px.choropleth_mapbox(df, geojson=dis_json, locations='id', 
                           color_continuous_scale="Viridis",
                           zoom=10, center = {"lat": 29.561, "lon": 78.663},
                           opacity=0.5,
                           labels={'Plot_No':'Plot Number'},
                           hover_data = ['Plot_No', 'UID']
                          )



fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "Google, RBS Pvt. Ltd.",
            "source": [
                "http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}"
            ]
        }
      ])

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update(layout_showlegend=False)

#------------------Search Overlay--------------------------------------------------------
area_list_options = [{'label': "All", 'value': "all"}]

for area in area_list:
    area_list_options.append({'label': area, 'value': area})

card_content_1 = [
    dbc.CardBody(
        [ 
            dbc.Row([dbc.Col(html.I(className="fas fa-search-location fa-2x", id = 'search-icon'), width = 1), 
                     dbc.Col(dcc.Dropdown(
                options=area_list_options,
                id = 'area_list_combo',
                className = 'search-combobox',
                value = "all"
            ), width = 10)]),
            
            
         
        ], className = 'searchbox'
    ),
]

card_search = dbc.Card(card_content_1, color="dark", inverse=True)

search_overlay_div = html.Div([
    
    card_search
    
    
    ] ,className = 'search-overlay')


#----------------------------------------------------------------------------------------

#-------------------Settings Overlay-----------------------------------------------------

settings_overlay_div = html.Div([
    
    dbc.Button(html.I(className = 'fas fa-sliders-h'), color = 'link', id = 'settings-collapse-button'),
    dbc.Tooltip("Basemap and Layer Settings",target = 'settings-collapse-button')
    ] ,className = 'settings-overlay')

settings_overlay_div_expanded = html.Div([
    
    dbc.Collapse([
            
            html.P('Basemap', className = 'overlay-form-labels'),
            dbc.Select(id = 'basemap-xyz-input', options = basemap_list_dict, value = "http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}", className = 'settings-inp'),
            html.P('Layer Opacity', className = 'overlay-form-labels'),
            dbc.Input(id = 'opacity-input', className = 'settings-inp', value = 0.5, type = 'number',min=0, max=1, step = 0.05),
            
            
        ], id = 'settings-exp')
    
    ] ,id = 'settings-overlay-expanded')

#----------------------------------------------------------------------------------------




mapfig = html.Div([search_overlay_div, settings_overlay_div, settings_overlay_div_expanded, dcc.Graph(figure = fig, style = {'height': '100%'}, id = 'basic-map')], className = 'map-first-page')

@app.callback(
    Output("settings-exp", "is_open"),
    [Input("settings-collapse-button", "n_clicks")],
    [State("settings-exp", "is_open")],
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


@app.callback(
    Output("basic-map", "figure"),
    [Input("area_list_combo", "value"), Input('basemap-xyz-input', 'value'), 
     Input('opacity-input', 'value')])
def load_basic_map(area, xyz, opac):
        
    area_list_df = pd.read_csv(area_list_csv_path)
    
    if(area == "all"):
        fig = px.choropleth_mapbox(df, geojson=dis_json, locations='id', 
                           color_continuous_scale="Viridis",
                           zoom=10, center = {"lat": 29.561, "lon": 78.663},
                           opacity=float(opac),
                           labels={'Plot_No':'Plot Number'},
                           hover_data = ['Plot_No', 'UID']
                          )



        fig.update_layout(
            mapbox_style="white-bg",
            mapbox_layers=[
                {
                    "below": 'traces',
                    "sourcetype": "raster",
                    "sourceattribution": "Google, RBS Pvt. Ltd.",
                    "source": [
                        xyz
                    ]
                }
              ])

        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.update(layout_showlegend=False)
        
        return fig
    else:
        fig = px.choropleth_mapbox(df, geojson=dis_json, locations='id', 
                           color_continuous_scale="Viridis",
                           zoom=area_list_df[area_list_df['label'] == area]['zoom'].values[0], 
                           center = {"lat": area_list_df[area_list_df['label'] == area]['lat'].values[0], 
                                              "lon": area_list_df[area_list_df['label'] == area]['lon'].values[0]},
                           opacity=float(opac),
                           labels={'Plot_No':'Plot Number'},
                           hover_data = ['Plot_No', 'UID']
                          )



        fig.update_layout(
            mapbox_style="white-bg",
            mapbox_layers=[
                {
                    "below": 'traces',
                    "sourcetype": "raster",
                    "sourceattribution": "Google, RBS Pvt. Ltd.",
                    "source": [
                        xyz
                    ]
                }
              ])

        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.update(layout_showlegend=False)
        
        return fig

    
#------------------Details Panel Content--------------------------------------

analysis_graphic = '...'

details_content = [
    dbc.CardHeader("Details"),
    dbc.CardBody(
        [
            html.Img(src = analysis_graphic, className = 'details-logo'),
            dbc.Alert("Click on any plot to see its details.", color="primary", style = {'border-radius': '5px'})
        ], id = 'basic-details-cardbody'
    ),
]


details_div = html.Div([dbc.Card(details_content, color="dark", inverse=True, style = {'height': '100%'})],id = 'details-basic-webmap')
#-----------------------------------------------------------------------------

@app.callback(Output('basic-details-cardbody', 'children'),
             [Input('basic-map', 'clickData')])
def basic_map_click(data_clicked):
    
    plot_details_df = pd.read_csv(plot_details_admin_file)
    
    if(data_clicked == None):
        return [html.Img(src = analysis_graphic, className = 'details-logo'), 
                dbc.Alert("Click on any plot to see its details.", color="primary", style = {'border-radius': '5px'})]
    else: 
        mini_df = plot_details_df[plot_details_df['UID'] == data_clicked['points'][0]['location']].dropna(axis='columns')
        
               
        
        if(len(mini_df) == 0):
            return [dbc.Alert("{}".format(data_clicked['points'][0]['location']), color="success", id = 'basic-details-header'),
                    dbc.Alert("No records found for the selected plot.".format(len(mini_df)), color="primary", style = {'border-radius': '5px'}),
                    ]
        else:
            
            details_table_body = []
        
            for c in range(len(mini_df.columns)):

                details_table_body.append(html.Tr([html.Td(mini_df.columns[c], className = 'basic-details-table-header'), html.Td(str(mini_df[mini_df['UID'] == data_clicked['points'][0]['location']][mini_df.columns[c]].values[0]),className = 'basic-details-table-content')]))
        
            
            return [dbc.Alert("{}".format(mini_df['UID'].values[0]), color="success", id = 'basic-details-header'),
                    dbc.Table(details_table_body, bordered=True, id = 'basic-details-table')]
    

# Layout for Admin page

In [8]:
admin_card_content_panel1 = [
    dbc.CardHeader("Select Industry/Park"),
    dbc.CardBody(
        [
            html.P('UID', id = 'admin-uid-label'),
            dcc.Dropdown(
                    id="uid-select",
                    options=[
                        {"label": x, "value": x} for x in plot_details_UID_list]
                ),
            dbc.Button('Open in Editor', color = 'primary', id = 'admin-open-editor-button'),
            
            dbc.Button('Show Dataset', color = 'success', id = 'admin-show-dataset-button', href = '/dataset'),
            
            dbc.Modal(
                [
                    dbc.ModalHeader("Changes Saved"),
                    dbc.ModalBody("Changes made to the dataset has been successfully saved."),
                    
                ],
                id="admin-alert-modal", centered = True
            ),
            
        ], className = 'admin-panel-1-card-body'
    ),
]
admin_panel_1 = dbc.Card(admin_card_content_panel1, color="dark", inverse=True, className = 'admin-panel-1-card')

admin_card_content_panel2 = [
    dbc.CardHeader("Record Editor"),
    dbc.CardBody(
        [], className = 'admin-panel-2-card-body', id = 'admin-panel-2-card-body'
    ),
]
admin_panel_2 = dbc.Card(admin_card_content_panel2, color="dark",  className = 'admin-panel-2-card', outline = True)

admin_card_content_panel3 = [
    dbc.CardHeader("Existing Status"),
    dbc.CardBody(
        [] , className = 'admin-panel-3-card-body', id = 'admin-panel-3-card-body'
    ),
]
admin_panel_3 = dbc.Card(admin_card_content_panel3, color="dark",  className = 'admin-panel-3-card')

## Layout Controls

In [9]:
app.layout = page_layout

#Routing
@app.callback([Output('navlinkscallback', 'children'),Output('mainareacallback', 'children')],
              [Input('url', 'pathname')])
def display_page(pathname):
    
    global is_logged_in
    
    if(pathname == "/"):
        
        if(is_logged_in):
        
            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'warning', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button(html.P(session_user_name, className = "col-12 text-truncate"), color = 'primary',
                               className = 'navlinkbutton', href = "/userinfo", id='username'),
                    dbc.Button([html.I(className = 'fas fa-sign-out-alt', style = {'margin-right': '5px'}), 'Logout'], color = 'danger',
                               className = 'navlinkbutton-logout', href = "/logout"),
                    dbc.Tooltip(session_user_name + " User Details", target = 'username', placement = 'bottom')
                    

                ], className = 'navlinkbuttongroup')
        
        else:

            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'warning', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login"),
                    
                    

                ], className = 'navlinkbuttongroup')
               
        main_area = [            
            dbc.Col(mapfig, width = 10, className = 'mainarea-map'),
            dbc.Col(details_div, width = 2, className = 'mainarea-details'),    
            
            ]
        
        return navigation_links, main_area
    
    elif(pathname == "/dis"):
        
        if(is_logged_in):
        
            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'warning', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button(html.P(session_user_name, className = "col-12 text-truncate"), color = 'primary',
                               className = 'navlinkbutton', href = "/userinfo", id='username'),
                    dbc.Button([html.I(className = 'fas fa-sign-out-alt', style = {'margin-right': '5px'}), 'Logout'], color = 'danger',
                               className = 'navlinkbutton-logout', href = "/logout"),
                    dbc.Tooltip(session_user_name + " User Details", target = 'username', placement = 'bottom')
                    

                ], className = 'navlinkbuttongroup')
        
        else:

            navigation_links = dbc.ButtonGroup([

                dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                dbc.Button('DIS Query', color = 'warning', className = 'navlinkbutton', href = '/dis'),
                dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login")
                    
                    

                ], className = 'navlinkbuttongroup')
        
        main_area = [            
                dbc.Col([
                    
                    html.H2('Page is Under Construction', className = 'text404'),
                    html.P('The page you are looking for is under construction.', className = 'desc404'),
                    html.Div(html.Img(src = "https://cdn.dribbble.com/users/285475/screenshots/2083086/dribbble_1.gif", className = 'image404'), className = 'box404'),
                    
                    
                ], width = 12),
            ]
        
        return navigation_links, main_area
    
    elif(pathname == "/admin"):
        
        if(is_logged_in):
        
            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'warning', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button(html.P(session_user_name, className = "col-12 text-truncate"), color = 'primary',
                               className = 'navlinkbutton', href = "/userinfo", id='username'),
                    dbc.Button([html.I(className = 'fas fa-sign-out-alt', style = {'margin-right': '5px'}), 'Logout'], color = 'danger',
                               className = 'navlinkbutton-logout', href = "/logout"),
                    dbc.Tooltip(session_user_name + " User Details", target = 'username', placement = 'bottom')
                    

                ], className = 'navlinkbuttongroup')
            
            main_area = [            
                    dbc.Col(admin_panel_1, width = 2, className = 'admin-panel-1'),
                    dbc.Col(admin_panel_2, width = 7, className = 'admin-panel-2'),
                    dbc.Col(admin_panel_3, width = 3, className = 'admin-panel-3')
                ]
        
        else:

            navigation_links = dbc.ButtonGroup([

                dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                dbc.Button('Administrator', color = 'warning', className = 'navlinkbutton', href = '/admin'),
                dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login")
                    
                    

                ], className = 'navlinkbuttongroup')
        
            main_area = [

                    dbc.Alert(
                        [
                            html.H4("You are not Logged In!!!", className="alert-heading"),
                            html.P(
                                "In order to read/write the database, You need to be in Administrator mode. Please Login to continue. "
                            ),
                            
                        ], color = 'danger', className = 'logout-info-div'
                    )
                ]
        
        return navigation_links, main_area
    
    elif(pathname == "/about"):
        
        if(is_logged_in):
        
            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'warning', className = 'navlinkbutton', href = "/about"),
                    dbc.Button(html.P(session_user_name, className = "col-12 text-truncate"), color = 'primary',
                               className = 'navlinkbutton', href = "/userinfo", id='username'),
                    dbc.Button([html.I(className = 'fas fa-sign-out-alt', style = {'margin-right': '5px'}), 'Logout'], color = 'danger',
                               className = 'navlinkbutton-logout', href = "/logout"),
                    dbc.Tooltip(session_user_name + " User Details", target = 'username', placement = 'bottom')
                    

                ], className = 'navlinkbuttongroup')
        
        else:

            navigation_links = dbc.ButtonGroup([

                dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                dbc.Button('About', color = 'warning', className = 'navlinkbutton', href = "/about"),
                dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login")
                    
                    

                ], className = 'navlinkbuttongroup')
        
        main_area = [            
                dbc.Col([
                    
                    html.H2('Page is Under Construction', className = 'text404'),
                    html.P('The page you are looking for is under construction.', className = 'desc404'),
                    html.Div(html.Img(src = "https://cdn.dribbble.com/users/285475/screenshots/2083086/dribbble_1.gif", className = 'image404'), className = 'box404'),
                    
                    
                ], width = 12),
            ]
        
        return navigation_links, main_area
    
    elif(pathname == "/login"):
        
        
        navigation_links = dbc.ButtonGroup([
        
                dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                dbc.Button('Login', color = 'warning', className = 'navlinkbutton-end', href = "/login")

            ], className = 'navlinkbuttongroup')
        
        email_input = dbc.FormGroup(
            [
                dbc.Label("Username", html_for="example-email"),
                dbc.Input(type="text", id="example-email", placeholder="Enter Username"),
                
            ]
        )

        password_input = dbc.FormGroup(
            [
                dbc.Label("Password", html_for="example-password"),
                dbc.Input(
                    type="password",
                    id="example-password",
                    placeholder="Enter Password",
                ),
                
            ]
        )
        
        submit_button = dbc.Button("Login", color="primary", style = {'width': '100px'}, id = 'login-button')
        
        card_content = [
            dbc.CardHeader("Login"),
            dbc.CardBody(
                [
                    dbc.Form([email_input, password_input, html.Div(submit_button, className = 'submitbutton')]),
                    
                    dbc.Modal(id="login-message", is_open = False, centered=True),
                    dbc.Button('Forgot Password', color = 'link', className = 'forgot-pass-button', href = '/forgot-password')
                    
                ]
            ),
        ]

        form = dbc.Card(card_content,color="dark", inverse=True, style = {'border-radius': '10px'})

        return navigation_links, html.Div(form, className = 'loginform')
    
    elif(pathname == "/logout"):
        
        
        
        is_logged_in = False
        
        navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login"),
                    
                    

                ], className = 'navlinkbuttongroup')
               
        main_area = [            
            
                    dbc.Alert(
                        [
                            html.H4("Logged Out Successfully!!!", className="alert-heading"),
                            html.P(
                                "You have been logged out successfully. If you wish to login again, click on Login on the top navigation. Alternatively, you can access the application in Guest mode. "
                            ),
                            
                        ], className = 'logout-info-div'
                    )
            
            ]
        
        return navigation_links, main_area
    
    elif(pathname == "/dataset"):
        
        
        if(is_logged_in):
            
            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button(html.P(session_user_name, className = "col-12 text-truncate"), color = 'primary',
                               className = 'navlinkbutton', href = "/userinfo", id='username'),
                    dbc.Button([html.I(className = 'fas fa-sign-out-alt', style = {'margin-right': '5px'}), 'Logout'], color = 'danger',
                               className = 'navlinkbutton-logout', href = "/logout"),
                    dbc.Tooltip(session_user_name + " User Details", target = 'username', placement = 'bottom')
                    

                ], className = 'navlinkbuttongroup')
        
            main_area = [

                        html.Div(dbc.Spinner([
                            
                            dbc.Toast(
                                    "Changes to the database has been recorded.",
                                    id="modal",
                                    header="Changes Saved",
                                    is_open=False,
                                    dismissable=True,
                                    icon="primary", 
                                    duration=4000,
                                    className = 'database-modal',
                                    style = {
                                        'position': 'fixed',
                                        'top': '85px',
                                        'right': '10px'}
                                ),
                            
                            dash_table.DataTable(
                                id='table',
                                columns=[{"name": i, "id": i} for i in plot_details_admin_df.columns],
                                data=plot_details_admin_df.to_dict('records'),
                                page_size = 50,
                                style_cell={
                                    'whiteSpace': 'normal',
                                    'height': 'auto', 'minWidth': '180px', 'width': '200px', 'maxWidth': '240px', 'textAlign': 'left'
                                },
                                style_table={'height': '500px', 'overflowY': 'auto', 'margin': '20px', 'width': 'auto',
                                             },

                                style_data = {'font-size': 'small', 'font-family': 'Roboto'},
                                style_header = {'font-size': 'small', 'font-family': 'Roboto'},
                                style_data_conditional=[
                                    {
                                        'if': {'row_index': 'odd'},
                                        'backgroundColor': 'rgb(248, 248, 248)'
                                    }
                                ],

                                editable=True,
                                filter_action="native",
                                sort_action="native",
                                sort_mode="multi",
                                selected_columns=[],
                                selected_rows=[],
                                page_action="native",
                                page_current= 0,
                                export_format='xlsx',
                                export_headers='display',
                                tooltip_data=[
                                    {
                                        column: {'value': str(value), 'type': 'markdown'}
                                        for column, value in row.items()
                                    } for row in plot_details_admin_df.to_dict('records')
                                ],

                            ),
                    
                    dbc.Button('Save', color = 'primary', id = 'savebutton', className = 'dataset-save'),
                                              
                    
                                             
                    ], type = 'grow', color = 'primary'), className = 'dataset-table-container',),

                    

                    

    
                ]
        else:
            
            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login"),
                    
                    

                ], className = 'navlinkbuttongroup')
            
            main_area = [

                    dbc.Alert(
                        [
                            html.H4("You are not Logged In!!!", className="alert-heading"),
                            html.P(
                                "In order to read/write the database, You need to be in Administrator mode. Please Login to continue. "
                            ),
                            
                        ], color = 'danger', className = 'logout-info-div'
                    )
                ]
        
        return navigation_links, main_area
        
    
    elif(pathname == "/userinfo"):
        
        if(is_logged_in):
        
            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button(html.P(session_user_name, className = "col-12 text-truncate"), color = 'warning',
                               className = 'navlinkbutton', href = "/userinfo", id='username'),
                    dbc.Button([html.I(className = 'fas fa-sign-out-alt', style = {'margin-right': '5px'}), 'Logout'], color = 'danger',
                               className = 'navlinkbutton-logout', href = "/logout"),
                    dbc.Tooltip(session_user_name + " User Details", target = 'username', placement = 'bottom')
                    

                ], className = 'navlinkbuttongroup')
            
            main_area = [
                
                            html.Div([
                                    
                                    dbc.Row([
                                            dbc.Col(html.H2([html.I(className="fas fa-user-tie 4x", id = 'userinfo-logo'),'User Details'], className = 'userinfo-h2'), width = 7),
                                            dbc.Col([dbc.Button('Change Password', color = 'primary', id = 'userinfo-change-pwd-button')], width = 5)
                                        ]),
                                    
                                    dbc.Row([dbc.Col(html.H3("{} {}".format(user_info_dict['First Name'], user_info_dict['Last Name'])))],
                                            className = 'userinfo-rows'),
                                
                                
                                ], className = 'userinfo-main-container')

                        
                    ]
        
        else:

            navigation_links = dbc.ButtonGroup([

                dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login")
                    
                    

                ], className = 'navlinkbuttongroup')
        
            main_area = [

                        dbc.Alert(
                            [
                                html.H4("You are not Logged In!!!", className="alert-heading"),
                                html.P(
                                    "In order to read/write the database, You need to be in Administrator mode. Please Login to continue. "
                                ),

                            ], color = 'danger', className = 'logout-info-div'
                        )
                    ]
        
        return navigation_links, main_area
    
    elif(pathname == '/forgot-password'):
        
        if(is_logged_in):
        
            navigation_links = dbc.ButtonGroup([

                    dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                    dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                    dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                    dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                    dbc.Button(html.P(session_user_name, className = "col-12 text-truncate"), color = 'warning',
                               className = 'navlinkbutton', href = "/userinfo", id='username'),
                    dbc.Button([html.I(className = 'fas fa-sign-out-alt', style = {'margin-right': '5px'}), 'Logout'], color = 'danger',
                               className = 'navlinkbutton-logout', href = "/logout"),
                    dbc.Tooltip(session_user_name + " User Details", target = 'username', placement = 'bottom')
                    

                ], className = 'navlinkbuttongroup')
            
            
        
        else:

            navigation_links = dbc.ButtonGroup([

                dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login")
                    
                    

                ], className = 'navlinkbuttongroup')
        
        main_area = [
                
                        html.Div([

                                dbc.Card([
                                    dbc.CardHeader('Forgot Password'),
                                    dbc.CardBody(['E-mail Address',
                                                    dbc.Input(type = 'email', id = 'email-input-forgot-pass'),
                                                    dbc.Button('Submit', id = 'send-verf-code-button', color = 'primary'),
                                                    html.Div(id='forgot-pass-alert')
                                                 ]),
                                    
                                ], className = 'forgot-pass-form', color = 'dark', inverse = True)


                            ], className = 'forgot-pass-container')
                    ]
        
        return navigation_links, main_area
    
    else:
        
        navigation_links = dbc.ButtonGroup([
        
                dbc.Button('Basic Web Map', color = 'secondary', className = 'navlinkbutton-start', href = '/'),
                dbc.Button('DIS Query', color = 'secondary', className = 'navlinkbutton', href = '/dis'),
                dbc.Button('Administrator', color = 'secondary', className = 'navlinkbutton', href = '/admin'),
                dbc.Button('About', color = 'secondary', className = 'navlinkbutton', href = "/about"),
                dbc.Button('Login', color = 'success', className = 'navlinkbutton-end', href = "/login")

            ], className = 'navlinkbuttongroup')
        
        main_area = [
            
                dbc.Col([
                    
                    html.H2('404: Looks like you are lost.', className = 'text404'),
                    html.P('The page you are looking for is not available.', className = 'desc404'),
                    html.Div(html.Img(src = "https://cdn.dribbble.com/users/285475/screenshots/2083086/dribbble_1.gif", className = 'image404'), className = 'box404'),
                    
                    
                ], width = 12),
            ]
        
        return navigation_links, main_area
        




## Callback for Dataset Page

In [10]:
@app.callback(
    [Output("modal", "is_open"), 
     Output('table', 'columns'), 
     Output('table', 'data'),
     Output('table', 'tooltip_data')],
    [Input("savebutton", "n_clicks")
     ],
    [State('table', 'data'), State('table', 'columns'), State("modal", "is_open")],
)
def toggle_modal(n1, rows, columns, is_open):
    
    plot_details_admin_df = pd.read_csv(plot_details_admin_file)
    for c in plot_details_admin_df.columns:
            if(c[:7] == 'Unnamed'):
                plot_details_admin_df = plot_details_admin_df.drop([c],axis = 1)
    
    if n1:    
        
        export_df = pd.DataFrame(rows, columns=[c['name'] for c in columns])
        
        
        export_df.to_csv(plot_details_admin_file)
        
        plot_details_admin_df = pd.read_csv(plot_details_admin_file)
        
        for c in plot_details_admin_df.columns:
            
            if(c[:7] == 'Unnamed'):
                plot_details_admin_df = plot_details_admin_df.drop([c],axis = 1)
          
        plot_details_admin_df.to_csv(plot_details_admin_file)
        
        tooltip_data=[
                {
                    column: {'value': str(value), 'type': 'markdown'}
                    for column, value in row.items()
                } for row in plot_details_admin_df.to_dict('records')
            ],

        
        
        return True, [{"name": i, "id": i} for i in plot_details_admin_df.columns], plot_details_admin_df.to_dict('records'), tooltip_data
    
    tooltip_data=[
                {
                    column: {'value': str(value), 'type': 'markdown'}
                    for column, value in row.items()
                } for row in plot_details_admin_df.to_dict('records')
            ],

        
    return False, [{"name": i, "id": i} for i in plot_details_admin_df.columns], plot_details_admin_df.to_dict('records'), tooltip_data



## Login Controls - User Authentication

In [11]:
@app.callback(Output('login-message', 'children'), Output('login-message', 'is_open'),
              [Input("login-button", "n_clicks")],
              [State('example-email', 'value'), State('example-password', 'value'), State('login-message', 'is_open')])
def autheticate_user(n, user, pwd, is_open):
    
    global is_logged_in
    global session_user_name
    global user_info_dict
    
    user_df = pd.read_csv(user_info_csv_path)
    
    if(n):
        
        mini_df = user_df[user_df['Username'] == user]
        
        if(len(mini_df) == 0):
            
            modal_content = [dbc.ModalHeader("Login Unsuccessful."),
                             dbc.ModalBody("No such user exists. Please check your inputs and try again.")]
            
            return modal_content, True
        else:
            if(mini_df['Password'].values[0] == pwd):
                
                modal_content = [dbc.ModalHeader("Login Successful."),
                             dbc.ModalBody("Welcome, {}".format(mini_df['First Name'].values[0])),
                             dbc.ModalFooter(dbc.Button("Okay", href="/", className="ml-auto"))]
                
                is_logged_in = True
                
                session_user_name = "{} {}".format(mini_df['First Name'].values[0], mini_df['Last Name'].values[0])
                
                user_info_dict = {'First Name': mini_df['First Name'].values[0],
                                  'Middle Name': mini_df['Middle Name'].values[0],
                                  'Last Name': mini_df['Last Name'].values[0],
                                  'DOB': mini_df['DOB'].values[0],
                                  'Username': mini_df['Username'].values[0],
                                  'Password': mini_df['Password'].values[0]}
                
                
            
                return modal_content, True
                
            else:
                modal_content = [dbc.ModalHeader("Login Unsuccessful."),
                             dbc.ModalBody("Invalid Password. Please check your inputs and try again.")]
            
                return modal_content, True


## Forgot Password Page Callbacks

In [12]:
@app.callback(Output('forgot-pass-alert', 'children'),
             [Input('send-verf-code-button', 'n_clicks')],
             [State('email-input-forgot-pass','value')])
def forgot_password(n, email):
    user_df = pd.read_csv(user_info_csv_path)
    
    if(n):
        mini_df = user_df[user_df['E-mail'] == email]
        
        if(len(mini_df) == 0):
            return dbc.Alert('The E-mail is not associated with any username. Try Again', color = 'danger', className = 'forgot-pass-alert-child')
        else:
            
            message = """\
            Subject: Forgot Password for UIIS
            
            Dear {},

            Password for the user {} is {}.
            
            RBS Pvt Ltd.""".format(mini_df['First Name'].values[0],mini_df['Username'].values[0],mini_df['Password'].values[0])


            # Create a secure SSL context
            context = ssl.create_default_context()

            with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
                server.login(sender_email, password)
                server.sendmail(sender_email, email, message)
            
            return dbc.Alert('The password has been sent to {}.'.format(email), color = 'success', className = 'forgot-pass-alert-child')

## Callback for Admin Page

In [13]:
selected_admin_uid = ''

@app.callback(Output('admin-panel-2-card-body', 'children'), Output('admin-panel-3-card-body', 'children'),
              Input('admin-open-editor-button', 'n_clicks'),
              State('uid-select', 'value'))
def load_admin_editor(n, uid):
    
    global selected_admin_uid
    
    selected_admin_uid = uid
    
    if(n):
        
        plot_details_admin_df = pd.read_csv(plot_details_admin_file)
        for c in plot_details_admin_df.columns:
            if(c[:7] == 'Unnamed'):
                plot_details_admin_df = plot_details_admin_df.drop([c],axis = 1)
                
        mini_admin_df = plot_details_admin_df[plot_details_admin_df['UID'] == uid]
        
        if(len(mini_admin_df) == 0):
            return1 = dcc.Loading([html.H1(uid),
                                dbc.Alert([dbc.Row([
                                                    dbc.Col(html.I(className = "fas fa-exclamation-circle 2x"),width = 1),
                                                    dbc.Col(html.P('No record found for the selected UID'), width = 11)
                                                ]),
                                           
                                          
                                          ], color = 'danger', className='record-unavailable-alert')
                               ])
            
            return2 = dcc.Loading([html.H3(uid),
                                dbc.Alert([dbc.Row([
                                                    
                                                    dbc.Col(html.P('No record found for the selected UID'), width = 12)
                                                ]),
                                           
                                          
                                          ], color = 'danger', className='record-unavailable-alert')
                               ])
            
            return return1, return2
        else:
            
            df_for_table = pd.DataFrame()
            df_for_table['Properties'] = mini_admin_df.columns
            df_for_table['Values'] = mini_admin_df.values[0]
            
            admin_data_table_editor = dash_table.DataTable(
                    id='admin-data-table-editor',
                    columns=[{"name": i, "id": i} for i in df_for_table.columns],
                    data=df_for_table.to_dict('records'),
                    style_cell={'textAlign': 'left',
                                'whiteSpace': 'normal',
                                'minWidth': '50%', 'width': '50%', 'maxWidth': '50%',
                                'height': 'auto',},
                    style_data = {'font-size': 'small', 'font-family': 'Roboto'},
                    style_header = {'font-size': 'small', 'font-family': 'Roboto', 'font-weight': 'bold'},
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ],

                    editable=True,
                    
                    tooltip_data=[
                        {
                            column: {'value': str(value), 'type': 'markdown'}
                            for column, value in row.items()
                        } for row in df_for_table.to_dict('records')
                    ],


                )
            
            return1 = dcc.Loading([dbc.Row([dbc.Col(html.H1(uid), width = 10, id = 'admin-uid-h1'), 
                                         dbc.Col(dbc.Button('Save Changes', color = 'primary', id = 'admin-save-changes-button'), width = 2)]),
                                html.Div(admin_data_table_editor, className = 'admin-table-container')
                               ])
            
            return2 = dcc.Loading([html.H3(uid),
                                html.Div(dbc.Table.from_dataframe(df_for_table, striped=True, bordered=True, hover=True), className = 'admin-existing-details-container')
                               ])
            
            return return1, return2

@app.callback(Output('admin-alert-modal', 'is_open'), 
             [Input('admin-save-changes-button', 'n_clicks')],
             [State('admin-data-table-editor', 'data'), 
              State('admin-data-table-editor', 'columns'), 
              State("admin-alert-modal", "is_open")])
def admin_save_changes(n, rows, columns, is_open):
    
    global selected_admin_uid
    
    if(n):
        export_df = pd.DataFrame(rows, columns=[c['name'] for c in columns])
        
        plot_details_admin_df = pd.read_csv(plot_details_admin_file)
        for c in plot_details_admin_df.columns:
            if(c[:7] == 'Unnamed'):
                plot_details_admin_df = plot_details_admin_df.drop([c],axis = 1)
        
        for c in plot_details_admin_df.columns:
            plot_details_admin_df.loc[plot_details_admin_df['UID'] == selected_admin_uid, c] = export_df[export_df['Properties'] == c]['Values'].values[0]
            
                           
        plot_details_admin_df.to_csv(plot_details_admin_file)
        
        return True

In [ ]:

if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/May/2021 13:22:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:33] "GET /assets/admin.css?m=1617688228.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:33] "GET /assets/dataset-page.css?m=1617688228.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:34] "GET /assets/login.css?m=1617688228.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:34] "GET /assets/logout.css?m=1617688228.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:34] "GET /assets/main.css?m=1617688228.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:34] "GET /assets/navbar.css?m=1617688228.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:34] "GET /assets/page.css?m=1617688228.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:34] "GET /assets/userinfo.css?m=1617688228.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:22:34] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_2m1602972844.13.0.min.js HTTP/1.1" 200 -
127.0.0

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Miniconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Miniconda3\lib\site-packages\dash\dash.py", line 1072, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Miniconda3\lib\site-pa

127.0.0.1 - - [05/May/2021 13:22:57] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [05/May/2021 13:23:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:09] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Miniconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Miniconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Miniconda3\lib\site-packages\dash\dash.py", line 1072, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Miniconda3\lib\site-pa

127.0.0.1 - - [05/May/2021 13:23:10] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [05/May/2021 13:23:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:13] "GET /_dash-component-suites/dash_table/async-table.v4_10_1m1599150397.js HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 13:23:55] "POST 